In [1]:
import os
import requests
import pandas as pd
import json

from dotenv import load_dotenv
load_dotenv()

%matplotlib inline

In [2]:
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'


token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')
params = {'token': token}
test_params = {'token': test_token}

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [3]:
# Functions that returns the chart of a single stock ticker 

# Test/sandbox function  
def call_test_chart_api(stock_ticker, chart_range):
    
    resp = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp.json())
    df.set_index('date',inplace=True)
    return df

# Real API function 
def call_real_chart_api(stock_ticker, chart_range):
    
    resp = requests.get(base_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+token)
    df = pd.DataFrame(resp.json())
    df.set_index('date',inplace=True)
    return df

In [4]:
# Create function that takes a list of tickers, calls the API, and returns a DataFrame
# with all close prices from the last 10 years of all tickers that were passed to the function 
def test_apicall_get_close_10_years(tickers_list):
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list: 
        
        if i == 0: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/10y?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())

            data = resp_df.set_index('date')
            combined_df_f = data[['close']]
            symbol = i 
            combined_df_f.rename(columns={'close':symbol},inplace=True)
            
        else: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/10y?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())
            
            data = resp_df.set_index('date')
            df_f = data[['close']]
            #symbol = data.iloc[0,4]
            symbol = i
            df_f.rename(columns={'close':symbol},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            
    return combined_df_f   

# Below function is same as function as above - only difference is it uses the real 
# IEX API tokens as opposed to the Sandbox/Test API tokens that IEX provides for testing your code

# IEX has limited free calls a month. It has lots of available calls, but there's no reason to 
# waste credits calling the real API while testing code. Should save real API calls for final product 

def real_apicall_get_close_10_years(tickers_list):
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list: 
        
        if i == 0: 
            resp_data = requests.get(base_url+'stock/'+i+'/chart/10y?token='+token)
            resp_df = pd.DataFrame(resp_data.json())

            data = resp_df.set_index('date')
            combined_df_f = data[['close']]
            symbol = i 
            combined_df_f.rename(columns={'close':symbol},inplace=True)
            
        else: 
            resp_data = requests.get(base_url+'stock/'+i+'/chart/10y?token='+token)
            resp_df = pd.DataFrame(resp_data.json())
            
            data = resp_df.set_index('date')
            df_f = data[['close']]
            #symbol = data.iloc[0,4]
            symbol = i
            df_f.rename(columns={'close':symbol},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            
    return combined_df_f  

In [5]:
tickers = ['GME','AMC','AMZN','WMT','NVDA','AMD','AAPL','MSFT','DIS','PYPL']

In [6]:
test_df = test_apicall_get_close_10_years(tickers)
test_df

C:\Users\watso\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
date,,,,,,,,,,
2011-09-26,24.07,NaN,238.50,52.80,3.5030,6.60,15.1155,25.640,30.40,NaN
2011-09-27,24.36,NaN,224.30,54.02,3.5300,6.72,14.5082,26.390,31.97,NaN
2011-09-28,23.66,NaN,230.84,52.41,3.3830,6.35,14.7572,26.273,31.85,NaN
2011-09-29,24.11,NaN,226.65,51.99,3.2965,5.34,14.5164,26.320,31.13,NaN
2011-09-30,23.90,NaN,226.78,53.90,3.1687,5.33,13.6519,25.210,31.27,NaN
...,...,...,...,...,...,...,...,...,...,...
2021-09-17,211.54,44.80,3580.69,146.43,227.0000,107.79,146.5600,302.740,190.26,288.69
2021-09-20,195.30,40.91,3468.85,144.60,220.3700,104.18,146.5700,298.500,187.02,270.06
2021-09-21,195.15,40.24,3382.87,150.15,215.7900,103.22,147.5800,298.300,174.76,271.68


In [7]:
# Can do whatever we want with the dataframe and close prices - but beware of dropping data.
# Not all stocks in our ticker list were publicly traded over 10 years ago.
# PayPal and AMC , which were still privately traded in 2011, do not show up as data and can 
#  mess up the DataFrame, as seen below with "dropna", where we lose ~5 years of data with other stocks.

daily_returns = test_df.pct_change()
daily_returns.dropna(inplace=True)
daily_returns

,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
date,,,,,,,,,,
2015-07-21,-0.003664,-0.022590,-0.030303,0.007570,0.018441,0.055556,-0.026554,-0.021109,0.020494,-0.022485
2015-07-22,-0.021251,0.013867,0.028649,-0.009885,-0.053544,-0.026316,-0.044010,-0.005920,-0.008179,-0.024981
2015-07-23,0.024843,0.027660,-0.022246,-0.021832,0.038509,-0.043243,0.036676,-0.014887,-0.017799,-0.046677
2015-07-24,-0.027704,-0.054718,0.066612,0.001497,0.007472,-0.039548,-0.023687,0.005397,0.034081,0.011176
2015-07-27,-0.026608,0.006571,0.038678,-0.008425,-0.016657,-0.011765,-0.007382,-0.003006,-0.038505,-0.005789
...,...,...,...,...,...,...,...,...,...,...
2021-09-17,-0.008577,-0.054254,0.007725,-0.002860,-0.014885,-0.031014,-0.052189,-0.051893,0.017705,0.012095
2021-09-20,-0.076770,-0.086830,-0.031234,-0.012497,-0.029207,-0.033491,0.000068,-0.014005,-0.017029,-0.064533
2021-09-21,-0.000768,-0.016377,-0.024786,0.038382,-0.020783,-0.009215,0.006891,-0.000670,-0.065554,0.005999


In [8]:
# Call a single stock's chart data , and provide time frame for data requested 
gme_chart_1 = call_test_chart_api('GME','1y')
gme_chart_2 = call_test_chart_api('GME','10y')

In [9]:
gme_chart_1

,close,high,low,open,symbol,volume,id,key,subkey,updated,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
date,,,,,,,,,,,,,,,,,,,,,
2020-09-24,9.30,10.1700,9.30,10.02,GME,8309615,THLACIISCPOSR_ERI,GEM,,1634211975668,...,9.40,8163414,9.98,9.24,9.9300,9.16,8312196,"Sep 24, 20",0.000000,0.0000
2020-09-25,10.06,10.3313,9.39,9.39,GME,7790583,CIRPL_EASCSIHTROI,GEM,,1681144661827,...,9.40,7675384,9.41,10.33,10.3393,9.50,7882779,"Sep 25, 20",0.919628,0.0981
2020-09-28,10.41,10.5400,10.00,10.54,GME,7078294,SCEOHLPARCI_RISIT,MEG,,1640836452307,...,9.86,6853471,10.21,10.42,10.6700,9.65,6976351,"Sep 28, 20",0.071556,0.0070
2020-09-29,10.46,10.8200,10.00,10.00,GME,5331860,AISRIILPOSHTCR_EC,GME,,1685416041106,...,10.03,5247047,10.00,10.79,10.7900,10.15,5399292,"Sep 29, 20",0.265436,0.0269
2020-09-30,10.60,10.9688,10.41,10.55,GME,6212194,CATSHSCRERI_LOIIP,GEM,,1667016987345,...,10.46,6109647,10.44,10.50,11.0744,10.34,6290472,"Sep 30, 20",-0.156613,-0.0146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-17,213.23,219.4700,208.88,218.20,GME,3989123,LRCITRSAE_IHICOSP,MGE,,1645174564611,...,209.90,4048313,216.85,210.87,214.5300,207.57,3980133,"Sep 17, 21",-1.449312,-0.0071
2021-09-20,199.10,206.0000,189.77,206.00,GME,4049232,AITIRSEOHR_ICLCSP,EGM,,1686045425949,...,187.85,4096986,201.00,196.90,209.9836,189.04,4005950,"Sep 20, 21",-12.791325,-0.0626
2021-09-21,197.18,203.7900,187.00,200.98,GME,2681534,IRLII_OESCPTCHARS,GEM,,1654811275382,...,193.00,2753893,202.38,197.00,202.3900,188.00,2749070,"Sep 21, 21",-2.280000,-0.0119


In [10]:
gme_chart_2

,close,high,low,open,symbol,volume,id,key,subkey,updated,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
date,,,,,,,,,,,,,,,,,,,,,
2011-09-26,24.18,24.79,22.64,23.73,GME,2131224,ICOIESCARSI_RTHPL,GME,,1617290479100,...,23.10,2097356,15.5850,16.4857,16.4112,15.2017,2164416,"Sep 26, 11",0.000000,0.0000
2011-09-27,24.20,24.83,24.20,24.83,GME,2681817,CHIEILI_PTCRSSAOR,GEM,,1677982564776,...,23.98,2569990,16.4744,16.7054,16.4530,15.9141,2645535,"Sep 27, 11",0.277259,0.0116
2011-09-28,23.74,24.66,23.74,24.56,GME,2368653,SII_RAOEHLTCCPISR,GME,,1660573246233,...,24.12,2429990,16.4690,15.8960,16.3607,15.8843,2363752,"Sep 28, 11",-0.367643,-0.0156
2011-09-29,24.54,25.02,23.91,24.50,GME,3120032,CROIIEAIC_SHSRPTL,EGM,,1660658365831,...,24.07,3094886,16.0257,16.7048,16.6695,16.2548,3100211,"Sep 29, 11",0.474626,0.0200
2011-09-30,23.90,24.15,23.71,23.71,GME,2832458,PTROIIL_ICHRSSAEC,GME,,1670681056694,...,23.10,2837666,16.1181,15.5460,16.4219,15.9361,2782607,"Sep 30, 11",-0.891855,-0.0381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-17,215.05,220.55,209.17,214.25,GME,4031960,APOT_ILRHSCEIRCSI,EGM,,1693508739283,...,201.42,4086861,211.9800,214.5100,217.6700,204.3900,4119634,"Sep 17, 21",-1.454167,-0.0071
2021-09-20,195.00,205.00,193.02,205.00,GME,4120305,CARITLISPROISCHE_,GME,,1636460230912,...,189.60,3947350,200.0000,197.4000,207.1869,193.3700,4042171,"Sep 20, 21",-12.997793,-0.0626
2021-09-21,192.77,206.60,186.00,206.60,GME,2723686,ERISPIATOLHCC_RSI,MGE,,1634467025661,...,189.00,2705914,208.8500,190.9300,199.9900,194.0000,2765330,"Sep 21, 21",-2.260000,-0.0123
